In [1]:
import numpy
import scipy.constants as sciConst
import sympy
from PIL import Image                   # View in default viewer

In [2]:
def getCartisianVectorValues(chargesCartesianCoordinates = []):
    if len(chargesCartesianCoordinates) <= 0:
        raise Exception("This function needs at least one dimension.")

    if len(chargesCartesianCoordinates) > 4:
        raise Exception("Tried to calculate too many dimensions.")

    unitVector = []
    magnitude = 0

    for component in chargesCartesianCoordinates:
        try:
            magnitude = numpy.sqrt(magnitude**2+component**2)
        except:
            magnitude = sympy.sqrt(magnitude**2+component**2)
    
    for component in chargesCartesianCoordinates:
        if magnitude == 0:
            unitVector.append(1)
        unitVector.append(component/magnitude)

    return [unitVector, magnitude]

In [3]:
charge = sciConst.elementary_charge # Unit: Coulombs (C)
permittivity = 8.854187 * 10**-12 # Unit: Farads per meter (F*m^-1)

fieldFormulaConstants = charge/(4*numpy.pi*permittivity)

In [4]:
def calculateFieldImages(xAmount = 1, yAmount = 1, zAmount = 1, cubeLength = 1, charges = []):

    scalingFactor = ((3/4)*10**8)*(cubeLength)**2/(10**-numpy.log10(cubeLength))

    xValues = numpy.linspace(-cubeLength,cubeLength,xAmount)
    yValues = numpy.linspace(cubeLength,-cubeLength,yAmount)
    zValues = numpy.linspace(-cubeLength,cubeLength,zAmount)

    images = []

    for imageZ in range(0,zAmount):
        points = numpy.zeros((xAmount,yAmount,4), dtype=numpy.uint8)
        for pixelX in range(0,xAmount):
            for pixelY in range(0,yAmount):
                composedUnitVector = [1,1,1]
                composedMagnitude = 0
                for charge in charges:
                    x = xValues[pixelX] - charge[0]
                    y = yValues[pixelY] - charge[1]
                    [r,m] = getCartisianVectorValues((x,y,zValues[imageZ]))
                    componentX = r[0]/m**3+composedUnitVector[0]*composedMagnitude
                    componentY = r[1]/m**3+composedUnitVector[1]*composedMagnitude
                    componentZ = r[2]/m**3+composedUnitVector[2]*composedMagnitude
                    vector = (componentX,componentY,componentZ)
                    [composedUnitVector,composedMagnitude] = getCartisianVectorValues(vector)
                for component in range(0,3):
                    points[pixelY,pixelX,component] = 127.5 * composedUnitVector[component] + 127.5
                
                effect = (scalingFactor * fieldFormulaConstants * composedMagnitude) * 100
            
                if effect > 255:
                    points[pixelY,pixelX,3] = 255
                elif effect < 0:
                    points[pixelY,pixelX,3] = 0
                else:
                    points[pixelY,pixelX,3] = effect

        images.append(points)
    
    return images

In [5]:
# images1 = calculateFieldImages(100,100,3,100,charges=[[0,0]])
# images2 = calculateFieldImages(100,100,3,1,charges=[[0,0]])
# images3 = calculateFieldImages(100,100,3,0.01,charges=[[0,0]])

# images=[images1,images2,images3]

chart = calculateFieldImages(200,200,31,10,charges=[[5,0],[-5,0],[0,5],[0,-5],[0,0]])

for plot in range(0,len(chart)):
    image = Image.fromarray(chart[plot], mode="RGBA")
    image.save("fieldImages\\%i.png"%(plot),"PNG")

# z1,z2,z3,z4,z5 = Image.fromarray(images[0]), Image.fromarray(images[1]), Image.fromarray(images[2]), Image.fromarray(images[3]), Image.fromarray(images[4])

# z1.show()
# z2.show()
# z3.show()
# z4.show()
# z5.show()